In [ ]:
# B T C = Batch Size, Sequence Length (in tokens), Embedding Dimension
# End in the same projection as before - 4, 16, 8
# Q K V = (8, 16) (16, 8) (8, 16)

In [12]:
import torch

q = torch.randn(4, 8, 16)
k = torch.randn(4, 16, 8)
v = torch.randn(4, 8, 16)

attention_scores = q @ k
output = attention_scores @ v
attention_scores.shape
output.shape

torch.Size([4, 8, 16])